In [1]:
import random

import matplotlib.animation as animation
from matplotlib.widgets import Slider, Button
import matplotlib as mpl
from matplotlib import pyplot as plt
import scipy.interpolate as inter
import numpy as np
import pandas as pd

from wsp import util
from wsp import ds
from wsp import tsp

QTREE = ds.PKPRQuadTree

In [2]:
data = [] # num of jumps, left points, right points
# MARK: SETUP
N_LEFT  = 6 # How many points in the left cluster
N_RIGHT = 1 # How many points in the right cluster
RADIUS_LEFT = 1.0
RADIUS_RIGHT = 1.0
OFFSET_RIGHT = 10.0

def calculate_jumps(old_path : list[ds.Point], new_path : list[ds.Point]) -> int:
    jumps = 0

    old_edges = {
        frozenset([old_path[i], old_path[i + 1]])
        for i in range(len(old_path) - 1)
    }
    new_edges = {
        frozenset([new_path[i], new_path[i + 1]])
        for i in range(len(new_path) - 1)
    }
    # print(old_edges - new_edges)

    return len(old_edges - new_edges)

In [3]:
def run_simulation() -> list:
    left_points : list[ds.Point] = util.generate_points(N_LEFT, lambda: (random.uniform(-RADIUS_LEFT, RADIUS_LEFT), random.uniform(-RADIUS_LEFT, RADIUS_LEFT))) # Generate points
    right_points : list[ds.Point] = util.generate_points(N_RIGHT, lambda: (random.uniform(-RADIUS_RIGHT, RADIUS_RIGHT) + OFFSET_RIGHT, random.uniform(-RADIUS_RIGHT, RADIUS_RIGHT)))
    # left_points = [ds.Point(0,1), ds.Point(0,-1), ds.Point(1,1), ds.Point(1,-1), ds.Point(0.25,0.25), ds.Point(1,0)]
    # right_points = [ds.Point(10,0)+ ds.Point(10,0.5)]

    ts_problem = tsp.TravellingSalesmanProblem[QTREE](QTREE, right_points + left_points, np.array([None, None]), s=2.0)
    left_problem = tsp.TravellingSalesmanProblem[QTREE](QTREE, left_points, np.array([None, None]), s=2.0)

    first_right_index = next(i for i in range(500) if ts_problem.dp_path[0][i] in right_points)
    

    new_solve_small = ts_problem.dp_path[0][first_right_index + N_RIGHT:]
    new_solve_small.extend(# REVIEW: we may not actually need this could save some time
        x
        for x in ts_problem.dp_path[0][:first_right_index]
        if x not in new_solve_small
    )
    new_solve_small = new_solve_small[:-1] if first_right_index == 0 else new_solve_small

    # ensure that the left points are all in a row
    # assert set(new_solve_small) == set(ts_problem.dp_path[0]) - set(ts_problem.dp_path[0][first_right_index:first_right_index + N_RIGHT - 1])
    assert len(new_solve_small) == len(left_problem.dp_path[0]) - 1
    jumps = calculate_jumps(left_problem.dp_path[0], new_solve_small)

    return jumps, left_points, right_points

# run_simulation()

In [7]:
for _ in range(100000):
    data.append(run_simulation())
len(data)

110000

In [8]:
df = pd.DataFrame(data, columns=["jumps", "left", "right"])
df

,jumps,left,right
0,1,"[P(0.9414, -0.1176), P(0.4793, 0.5514), P(0.50...","[P(9.4173, -0.9614)]"
1,1,"[P(-0.7610, 0.3504), P(-0.7581, 0.5820), P(-0....","[P(10.5055, -0.7254)]"
2,1,"[P(0.2568, -0.6233), P(-0.2902, -0.2964), P(0....","[P(10.8184, -0.0149)]"
3,2,"[P(-0.8721, 0.1627), P(0.7083, -0.4790), P(-0....","[P(10.7252, 0.1938)]"
4,1,"[P(-0.0257, 0.5587), P(-0.5345, 0.7979), P(0.4...","[P(10.0636, -0.7690)]"
...,...,...,...
109995,1,"[P(0.0552, -0.0362), P(0.5320, 0.4421), P(0.76...","[P(10.9023, -0.6240)]"
109996,1,"[P(0.6686, -0.0027), P(-0.2754, 0.3664), P(0.2...","[P(10.7395, -0.0824)]"
109997,1,"[P(0.5684, 0.3576), P(-0.3191, 0.5133), P(0.37...","[P(10.6893, -0.3115)]"
109998,1,"[P(0.3269, 0.1085), P(-0.5816, -0.4247), P(-0....","[P(9.8634, 0.3535)]"


In [9]:
df["jumps"].value_counts()

jumps
1    77667
2    22571
3     8619
4     1143
Name: count, dtype: int64